In [1]:
from Hamil_search import *
from simulation import *
import numpy as np
import os
import pennylane as qml

In [2]:
def load_or_precompute(n_block):
    gx, gz = 1, 3
    if not os.path.exists(f"Hpen_{n_block}blocks.npy"):
        n_phys = 4 * n_block
        Hpen_terms = []
        for i in range(n_block):
            Hpen_terms += [(gx, f'X{4*i}*X{4*i+1}'), (gz, f'Z{4*i}*Z{4*i+1}'),
                        (gx, f'X{4*i+2}*X{4*i+3}'), (gz, f'Z{4*i+2}*Z{4*i+3}')]
        Hpen = blocks2Mat(n_phys, Hpen_terms)
        P0, HpenInverse = get_P0_HpenInverse(Hpen)
        Uenc = getU(4, n_block)
        Penc = Uenc @ Uenc.conj().T
        np.save(f'Hpen_{n_block}blocks', Hpen)
        np.save(f'HpenInverse_{n_block}blocks', HpenInverse)
        np.save(f'P0_{n_block}blocks', P0)
        np.save(f'Uenc_{n_block}blocks', Uenc)
        np.save(f'Penc_{n_block}blocks', Penc)
    
    Hpen = np.load(f'Hpen_{n_block}blocks.npy')
    HpenInverse = np.load(f'HpenInverse_{n_block}blocks.npy')
    P0 = np.load(f'P0_{n_block}blocks.npy')
    Uenc = np.load(f'Uenc_{n_block}blocks.npy')
    Penc = np.load(f'Penc_{n_block}blocks.npy')
    
    return Hpen, HpenInverse, P0, Uenc, Penc

def test_leakage_and_get_logical_interaction(HpenInverse, P0, Penc, Uenc, Henc):
    A = Henc @ HpenInverse @ Henc
    off_diag = (P0 - Penc) @ A @ Penc
    if checkSame(off_diag, np.zeros(off_diag.shape)):
        print("no leakage")
    else:
        print("yes leakage")
    Hlogi = - Uenc.conj().T @ A @ Uenc
    return qml.pauli_decompose(Hlogi, hide_identity=True)
    

def get_P0_HpenInverse(Hpen):
    """
    Make sure that the eigenvalues of Hpen are all integers
    """
    e, u = np.linalg.eigh(Hpen)

    a = np.array([1 if abs(x) < 0.5 else 0 for x in e])
    P0 = u @ (np.expand_dims(a, axis=1) * u.conj().T)

    b = np.array([1/x if abs(x) > 0.5 else 0 for x in e])
    HpenInverse = u @ (np.expand_dims(b, axis=1) * u.conj().T)

    return P0, HpenInverse


# 2D TFIM

In [5]:
n_row = 1 # number of rows of blocks
n_col = 3 # number of columns of blocks

def idx(c, r, i):
    """
    Return the index of the physical qubit at the `i`-th position of the block located 
    at column `c` and row `r`.
    `c`, `r`, and the returned index all start from 0.
    `i` takes value from {1,2,3,4}.
    """
    assert 0 <= c and c < n_col
    assert 0 <= r and r < n_row
    assert 1 <= i and i <= 4
    return c * n_row * 4 + r * 4 + i - 1

n_block = n_row * n_col
n_phys = 4 * n_block
dim_phys = 2 ** n_phys

gx_flavors = [1, -1, 2]
gz_flavors = [3, -3, 4]

Hpen_terms = []
for c in range(n_col):
    for r in range(n_row):
        gx, gz = gx_flavors[r % 3], gz_flavors[r % 3]
        Hpen_terms += [(gx, f'X{idx(c, r, 1)}*X{idx(c, r, 2)}'), (gz, f'Z{idx(c, r, 1)}*Z{idx(c, r, 2)}'),
                       (gx, f'X{idx(c, r, 3)}*X{idx(c, r, 4)}'), (gz, f'Z{idx(c, r, 3)}*Z{idx(c, r, 4)}')]
Hpen = blocks2Mat(n_phys, Hpen_terms)
P0, HpenInverse = get_P0_HpenInverse(Hpen)
Uenc = getU(4, n_block)
Penc = Uenc @ Uenc.conj().T

Henc_terms = []
for c in range(n_col):
    for r in range(n_row):
        if r < n_row - 1: # vertical cross-block interactions
            Henc_terms += [(1, f'X{idx(c, r, 3)}*Z{idx(c, r+1, 2)}'), (1, f'X{idx(c, r, 3)}*Z{idx(c, r+1, 4)}')]
            Henc_terms += [(1, f'Z{idx(c, r, 2)}*X{idx(c, r+1, 3)}'), (1, f'Z{idx(c, r, 4)}*X{idx(c, r+1, 3)}')]
        if c < n_col - 1: # horizontal cross-block interactions
            if r % 2 == 0:
                Henc_terms += [(1, f'Z{idx(c, r, 2)}*X{idx(c+1, r, 3)}'), (1, f'Z{idx(c, r, 4)}*X{idx(c+1, r, 3)}')]
            else:
                Henc_terms += [(1, f'X{idx(c, r, 3)}*Z{idx(c+1, r, 2)}'), (1, f'X{idx(c, r, 3)}*Z{idx(c+1, r, 4)}')]

Henc = blocks2Mat(n_phys, Henc_terms)
test_leakage_and_get_logical_interaction(HpenInverse, P0, Penc, Uenc, Henc)

no leakage


(
    -0.3749999999999998 * Z(4)
  + 0.3749999999999998 * (Z(3) @ Z(4))
  + -0.3749999999999998 * Z(2)
  + 0.37499999999999983 * (Z(1) @ Z(2))
)

# Other

In [5]:
n_block = 2
n_phys = 4 * n_block
dim_phys = 2 ** n_phys

gx = [1, 2]
gz = [3, 4]

Hpen_terms = [(gx[0], 'X0*X1'), (gz[0], 'Z0*Z1'), (gx[0], 'X2*X3'), (gz[0], 'Z2*Z3'),
              (gx[1], 'X4*X5'), (gz[1], 'Z4*Z5'), (gx[1], 'X6*X7'), (gz[1], 'Z6*Z7')]
Hpen = blocks2Mat(n_phys, Hpen_terms)
P0, HpenInverse = get_P0_HpenInverse(Hpen)
Uenc = getU(4, n_block)
Penc = Uenc @ Uenc.conj().T

Henc_terms = [(1, 'X1*X6'), (1, 'X3*X6')]

Henc = blocks2Mat(n_phys, Henc_terms)
test_leakage_and_get_logical_interaction(HpenInverse, P0, Penc, Uenc, Henc)

no leakage


(
    -0.5714285714285714 * Z(2)
  + -0.5714285714285714 * (X(0) @ Z(2))
)

In [8]:
n_block = 2
n_phys = 4 * n_block
dim_phys = 2 ** n_phys

Hpen_terms = [(1, 'X0*X1'), (3, 'Z0*Z1'), (1, 'X2*X3'), (3, 'Z2*Z3'),
              (1, 'X4*X5'), (3, 'Z4*Z5'), (1, 'X6*X7'), (3, 'Z6*Z7')]
Hpen = blocks2Mat(n_phys, Hpen_terms)
P0, HpenInverse = get_P0_HpenInverse(Hpen)
Uenc = getU(4, n_block)
Penc = Uenc @ Uenc.conj().T

Henc_terms = [(1, 'Z1*X6'), (1, 'Z3*X6')]

Henc = blocks2Mat(n_phys, Henc_terms)

print(np.linalg.norm(P0 @ Henc @ HpenInverse @ Henc @ HpenInverse @ Henc @ Penc, ord=2))
test_leakage_and_get_logical_interaction(HpenInverse, P0, Penc, Uenc, Henc)

4.2896980535343984e-16
no leakage


(
    -0.3749999999999998 * Z(2)
  + 0.3749999999999998 * (Z(1) @ Z(2))
)

In [9]:
n_block = 3
n_phys = 4 * n_block
dim_phys = 2 ** n_phys

Hpen_terms = [(1, 'X0*X1'), (3, 'Z0*Z1'), (1, 'X2*X3'), (3, 'Z2*Z3'),
              (1, 'X4*X5'), (3, 'Z4*Z5'), (1, 'X6*X7'), (3, 'Z6*Z7'),
              (1, 'X8*X9'), (3, 'Z8*Z9'), (1, 'X10*X11'), (3, 'Z10*Z11')]
Hpen = blocks2Mat(n_phys, Hpen_terms)
P0, HpenInverse = get_P0_HpenInverse(Hpen)
Uenc = getU(4, n_block)
Penc = Uenc @ Uenc.conj().T

Henc_terms = [(1, 'Z1*X6'), (1, 'Z3*X6'),
              (1, 'Z5*X10'), (1, 'Z7*X10')]

Henc = blocks2Mat(n_phys, Henc_terms)

print(np.linalg.norm(P0 @ Henc @ HpenInverse @ Henc @ HpenInverse @ Henc @ Penc, ord=2))
test_leakage_and_get_logical_interaction(HpenInverse, P0, Penc, Uenc, Henc)

1.7261700700314458e-15
no leakage


(
    -0.3749999999999998 * Z(4)
  + 0.3749999999999998 * (Z(3) @ Z(4))
  + -0.3749999999999998 * Z(2)
  + 0.37499999999999983 * (Z(1) @ Z(2))
)

In [11]:
print(np.linalg.norm(P0 @ Henc @ HpenInverse @ Henc @ HpenInverse @ Henc @ P0, ord=2))

2.2244932225005978e-15
